In [ ]:
import matplotlib.pyplot as plt
import torch 

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import random
from torch.autograd import Variable
import torch.utils.data as Data
import pandas as pd
import torchvision

from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset


In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [ ]:
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True # keep True if all the input have same size.
SEED=42
seed_everything(SEED=SEED)

In [ ]:
transform1 = transforms.Compose(
    [transforms.Resize((256, 256)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform2 = transforms.Compose(
    [transforms.Resize((256, 256)),
     transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(degrees=15),
     #transforms.RandomCrop(256,256),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [ ]:
#train_data = torchvision.datasets.ImageFolder('../input/caltech101/Caltech101/train',transform=transform2)
train_data = torchvision.datasets.ImageFolder('../input/caltech101/Caltech101/train',transform=transform2)
eval_data=torchvision.datasets.ImageFolder('../input/caltech101/Caltech101/eval',transform=transform1)
test_data=torchvision.datasets.ImageFolder('../input/caltech101/Caltech101/test',transform=transform1)

train_loader=Data.DataLoader(dataset=train_data,batch_size=64,
                             shuffle=True, num_workers=2)
eval_loader=Data.DataLoader(dataset=eval_data,batch_size=128,
                             shuffle=True, num_workers=2)
test_loader=Data.DataLoader(dataset=test_data,batch_size=128,
                             shuffle=True, num_workers=2)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.conv1=nn.Sequential(
            nn.Conv2d(
                in_channels=3,out_channels=64,kernel_size=3,stride=1,
                padding=(1,1)  # if stride=1, padding=(kernel_size-1)/2
            ), 
            nn.ReLU(),            
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2)
        )
        self.conv2_1=nn.Sequential(
            nn.Conv2d(64,64,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv2=nn.Sequential(
            nn.Conv2d(64,128,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )

        
        self.conv3=nn.Sequential(
            nn.Conv2d(128,256,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2)
        )   
        self.conv4=nn.Sequential(
            nn.Conv2d(256,256,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2)
        )
        self.conv5=nn.Sequential(
            nn.Conv2d(256,128,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )
        
        self.dense1=nn.Sequential(
            nn.Linear(2048,1024),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.dense2=nn.Sequential(
            nn.Linear(1024,512),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        
        self.dense3=nn.Sequential(
            nn.Linear(512,256),
            nn.Dropout(0.5),
            nn.ReLU()
        )
       
        self.out=nn.Linear(256,101)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2_1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=self.conv5(x)
        x=x.view(x.size(0),-1)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        output=self.out(x)

        return output

cnn = CNN().cuda()
print(cnn)  # net architecture

In [ ]:
EPOCH = 140
BATCH_SIZE=32
LR=0.001
stepsize=40
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
optimizer=torch.optim.Adam(cnn.parameters(),lr=LR,weight_decay=1e-5)
#scheduler = ReduceLROnPlateau(optimizer, mode="min")
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1)
loss_func=nn.CrossEntropyLoss()



In [ ]:
#stepsize loss and epoch accuracy
iters = len(train_loader)
losses=[]
train_losses=[]
eval_accs=[]
train_accs=[]
stepsize=60
patience=4
early_stopping = EarlyStopping(patience=patience, verbose=True)
for epoch in range(EPOCH):
    
    cnn.train()
    train_acc=[]
    for step,(x,y) in enumerate(train_loader):
        b_x=Variable(x).cuda()
        b_y=Variable(y).cuda()
        output=cnn(b_x)
        loss=loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        loss2=loss_func(output,b_y)
        pred_y=torch.max(output,1)[1].cuda().data.squeeze()
        accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
        train_acc.append(accuracy)
        train_losses.append(loss2.data)
        if step%stepsize==0:
            losses.append(loss.data)
        scheduler.step(epoch + step/ iters)#######
    train_accs.append(np.mean(train_acc))
    torch.cuda.empty_cache()
    cnn.eval()
    with torch.no_grad():
        eval_acc=[]
        eval_loss=[]
        
    
        for x,y in eval_loader:
            
            b_x=Variable(x).cuda()
            b_y=Variable(y).cuda()        
            output=cnn(b_x)
            loss1=loss_func(output,b_y)
            pred_y=torch.max(output,1)[1].cuda().data.squeeze()
            accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
            eval_acc.append(accuracy)
            #scheduler.step(loss1)###############
            eval_loss.append(loss1.data)
        eval_accs.append(np.mean(eval_acc))
        
        
    
    print('Epoch: ',epoch,'| loss: ',loss.data,'| Eval_acc: ', np.mean(eval_acc),'| Train_acc: ', train_accs[-1])
   

In [ ]:
for epoch in range(40,80):
    
    cnn.train()
    train_acc=[]
    for step,(x,y) in enumerate(train_loader):
        b_x=Variable(x).cuda()
        b_y=Variable(y).cuda()
        output=cnn(b_x)
        loss=loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        loss2=loss_func(output,b_y)
        pred_y=torch.max(output,1)[1].cuda().data.squeeze()
        accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
        train_acc.append(accuracy)
        train_losses.append(loss2.data)
        if step%stepsize==0:
            losses.append(loss.data)
        scheduler.step(epoch + step/ iters)################
    train_accs.append(np.mean(train_acc))
    torch.cuda.empty_cache()
    cnn.eval()
    with torch.no_grad():
        eval_acc=[]
        eval_loss=[]
        
    
        for x,y in eval_loader:
            
            b_x=Variable(x).cuda()
            b_y=Variable(y).cuda()        
            output=cnn(b_x)
            loss1=loss_func(output,b_y)
            pred_y=torch.max(output,1)[1].cuda().data.squeeze()
            accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
            eval_acc.append(accuracy)
            #scheduler.step(loss1)#############
            eval_loss.append(loss1.data)
        eval_accs.append(np.mean(eval_acc))
        
        
    
    print('Epoch: ',epoch,'| loss: ',loss.data,'| Eval_acc: ', np.mean(eval_acc),'| Train_acc: ', train_accs[-1])
   

In [ ]:
for epoch in range(80,100):
    
    cnn.train()
    train_acc=[]
    for step,(x,y) in enumerate(train_loader):
        b_x=Variable(x).cuda()
        b_y=Variable(y).cuda()
        output=cnn(b_x)
        loss=loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        loss2=loss_func(output,b_y)
        pred_y=torch.max(output,1)[1].cuda().data.squeeze()
        accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
        train_acc.append(accuracy)
        train_losses.append(loss2.data)
        if step%stepsize==0:
            losses.append(loss.data)
        #scheduler.step(epoch + step/ iters)
    train_accs.append(np.mean(train_acc))
    torch.cuda.empty_cache()
    cnn.eval()
    with torch.no_grad():
        eval_acc=[]
        eval_loss=[]
        
    
        for x,y in eval_loader:
            
            b_x=Variable(x).cuda()
            b_y=Variable(y).cuda()        
            output=cnn(b_x)
            loss1=loss_func(output,b_y)
            pred_y=torch.max(output,1)[1].cuda().data.squeeze()
            accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
            eval_acc.append(accuracy)
            #scheduler.step(loss1)
            eval_loss.append(loss1.data)
        eval_accs.append(np.mean(eval_acc))
        
        
    
    print('Epoch: ',epoch,'| loss: ',loss.data,'| Eval_acc: ', np.mean(eval_acc),'| Train_acc: ', train_accs[-1])
   

In [ ]:
for epoch in range(120,140):
    
    cnn.train()
    train_acc=[]
    for step,(x,y) in enumerate(train_loader):
        b_x=Variable(x).cuda()
        b_y=Variable(y).cuda()
        output=cnn(b_x)
        loss=loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        loss2=loss_func(output,b_y)
        pred_y=torch.max(output,1)[1].cuda().data.squeeze()
        accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
        train_acc.append(accuracy)
        train_losses.append(loss2.data)
        if step%stepsize==0:
            losses.append(loss.data)
        scheduler.step(epoch + step/ iters)#################
    train_accs.append(np.mean(train_acc))
    torch.cuda.empty_cache()
    cnn.eval()
    with torch.no_grad():
        eval_acc=[]
        eval_loss=[]
        
    
        for x,y in eval_loader:
            
            b_x=Variable(x).cuda()
            b_y=Variable(y).cuda()        
            output=cnn(b_x)
            loss1=loss_func(output,b_y)
            pred_y=torch.max(output,1)[1].cuda().data.squeeze()
            accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
            eval_acc.append(accuracy)
            #scheduler.step(loss1)##############
            eval_loss.append(loss1.data)
        eval_accs.append(np.mean(eval_acc))
        
        
    
    print('Epoch: ',epoch,'| loss: ',loss.data,'| Eval_acc: ', np.mean(eval_acc),'| Train_acc: ', train_accs[-1])

In [ ]:
x=np.arange(len(losses))*stepsize
plt.plot(x,losses)
plt.title('loss ')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.show()

In [ ]:

x=np.arange(len(train_accs))
plt.plot(x,train_accs,label='train')
plt.plot(x,eval_accs,label='validate')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()